# ADC Simulation
Version: 12.2020

Mit dieser Simulation sollen grundsätzlich RunUp und RunDown Funktionen

# Library laden


In [1]:
import SimADC_V1 as adc
import pandas as pd
import plotly.graph_objects as px

import plotly.io as pio
pio.renderers.default = 'iframe'


## Simulation starten


In [2]:
print('Simulation startet')

# https://tinyurl.com/y9j2ls63

# 100k bei 0,1% = 100R bei 0,01% = 10R
# 2.5042
integrator = adc.Integrator()
integrator.Vin = 6.0

integrator.VrefN = -7.0458
integrator.VrefP =  7.0448
integrator.ChargingInjection = 100.0 # pC 

integrator.R_VrefN = 20.0  # [kOhm]
integrator.R_VrefP = 20.0  # [kOhm]
integrator.R_Vin =   40.0  # [kOhm]
integrator.C = 2.2 # in nF

cpu = adc.CPU()


# CPU Ticks = 0.2 us => 5 Mhz
# Anzahl der Rampen ( PLC 60Hz = 16,6ms  PLC 50Hz = 20 ms)
# Schalterwechsel mit einem Impuls von 200 ns ( 1/ 16)
cpu.RunUpCycles = 6250  # (6250 * 10 us = 62.5 ms = 50 Hz)

cpu.RunUpCnt    = 50    # => 10us
cpu.WaveForm    = 0


# Auflösung: 6250 * 200 = 1.250.000
# Bei Vin +/-10V = 20V / Aufslösung = 10 uV (theoretisch) 


cpu.Connect( integrator) # Kontrolle für den Integrator an die CPU übergeben
cpu.ResidueADC = adc.ResidueADC(cpu, 12, 3.0) # Reste ADC mit n bit erzeugen

# Kalibration Array 
# Parameter: Vadc Vin gemessen ADC
# Parameter: Vtrue Vin real
calibration_points = [
    (-6.064076321123799, -6.0),  # -Vref
    (-0.06420449762620192, 0.0),            # 0 V
    (5.935670086298077  ,   6.0)    # +Vref
]

# Initialisiere die Kalibrierung
#cpu.Calibration = adc.ADCCalibration(calibration_points)



cpu.Reset()
#cpu.AutoZero()
cpu.RunUp()
cpu.RunDown()
cpu.CalcVin()
#cpu.CalcVinX()







Simulation startet


## Diagramm erstellen
Vout am Integrator

Beipiele und Dokumention finden sie auf der folgenden Seite:
https://plotly.com/python/<br>
https://plotly.com/python/bar-charts/#basic-bar-chart-with-plotlygraphobjects

In [3]:
In
x_vals = [ float(i.Cnt) for i in integrator.RunUpCells]
y_vals = [ float(i.Value) for i in integrator.RunUpCells]

#print(x_vals)


# neues Diagramm erstellen mit Titel
fig = px.Figure()


# Linie anzeigen
fig.add_trace( px.Scatter( x=x_vals, y=y_vals , name="Vout" )  )

fig.update_layout(
   title_text='Multi-Slope (Runup)',
    yaxis_title="V",
    xaxis_title="Clock",
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)



fig.show()




In [4]:
print("Vin: ", integrator.Vin, " V")
print("Vref: ", integrator.VrefP, " V")
print("Vout", integrator.Vout, " V")
print("RunUp-Time: ", cpu.RunUpTime() , " ms")
print("RunDown-Count :", cpu.RunDownCnt *  0.1 , "us (Ticks)")
print("RunDown-Count :", cpu.RunDownCnt )
print("Vin berechnet: ", cpu.Vin , " V")
print("Vout-Before : ", cpu.Vout_Before , " V")
print("SIGN  : ", cpu.RunDown_Sign , " bit")



print("calc Vin V1  : ", cpu.V1 , " V")
print("calc Vin V2  : ", cpu.V2 , " V")
print("CntN :", cpu.CntN )
print("CntP :", cpu.CntP )
print("VAutoZero : mV", cpu.VAutoZero  ) 
print("Resolution uV:", cpu.Resolution * 1000.0 * 1000.0)


Vin:  6.0  V
Vref:  7.0448  V
Vout 0.9003181818177803  V
RunUp-Time:  62.49999999999999  ms
RunDown-Count : 0.0 us (Ticks)
RunDown-Count : 0
Vin berechnet:  5.8316867555  V
Vout-Before :  0.0  V
SIGN  :  0  bit
calc Vin V1  :  -5.834221568  V
calc Vin V2  :  0.0025348125000000006  V
CntN : 1831
CntP : 4419
VAutoZero : mV 0.0
Resolution uV: 0.0
